In [2]:
from Preprocessing.preprocessing_sorted import PreprocessingTrainingData
import seaborn as sns
import matplotlib.pyplot as  plt
import os
import logging
import pandas as pd
import pickle
#import midi_functions
import numpy as np

from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras import callbacks
from keras import optimizers

ModuleNotFoundError: No module named 'torch'

In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
#static parameters
train_batch_size = 30
val_batch_size = 30
sequence_length=50
test_batch_size = 1
input_size = 1
hidden_size = 256
num_layer = 2
output_size = 38
clip = 3

In [ ]:
#get data from preprocessing.py
dataset_path = os.path.join(os.path.abspath('..'),'Dataset\\Clementi dataset\\Clementi dataset' )
network_input,network_output,max_midi_number,min_midi_number,int_to_note = PreprocessingTrainingData().preprocess_notes(dataset_path)
#network_input, network_output = network_input.cuda(), network_output.cuda()
#max_midi_number
# print(network_input)
#print(network_output)

In [ ]:
input_length = 50
X_orig = network_input.numpy()
Y_orig = network_output.numpy()
#sorted_notes = np.loadtxt('notes/norm_sorted_notes.txt')
sorted_notes = np.unique(Y_orig)
n = X_orig.shape[0]

Y = network_output.numpy()
max_note = max_midi_number
min_note = min_midi_number

X = X_orig #normalize input
n_notes_Y = n_hidden_units = len(sorted_notes)

for i,note in enumerate(Y): #make all the notes a number between 0 and number_of_different_notes
    Y[i] = np.where(sorted_notes == note)[0]
Y = np_utils.to_categorical(Y)

print('X={}, Y={}'.format(len(np.unique(X)),n_notes_Y))
print('Shape of X: {}'.format(X.shape))
print('Max: {}, Min: {}'.format(max_note, min_note))

In [ ]:
#max_midi_number

In [ ]:
# network_input = pd.read_csv('Preprocessing\\network_input.csv')
# network_output = pd.read_csv('Preprocessing\\network_output.csv')

# network_input = np.asarray(network_input.values.tolist())
# network_input = torch.Tensor(network_input.reshape(network_input.shape[0], network_input.shape[1],1)).cuda()

# network_output = np.array(network_output.values.tolist())
# network_output = torch.Tensor(network_output.reshape(network_output.shape[0], )).cuda()

# max_midi_number = 105
# min_midi_number = 24

# # write python dict to a file
# # output = open('int_to_note.pkl', 'wb')
# # pickle.dump(int_to_note, output)
# # output.close()

# # read python dict back from the file
# # pkl_file = open('int_to_note.pkl', 'rb')
# # int_to_note = pickle.load(pkl_file)
# # pkl_file.close()

# int_to_note = {0: 24, 1: 26, 2: 27, 3: 28, 4: 29, 5: 30, 6: 31, 7: 32, 8: 33, 9: 34, 10: 35, 11: 36, 12: 37, 13: 38, 14: 39, 15: 40, 16: 41, 17: 42, 18: 43, 19: 44, 20: 45, 21: 46, 22: 47, 23: 48, 24: 49, 25: 50, 26: 51, 27: 52, 28: 53, 29: 54, 30: 55, 31: 56, 32: 57, 33: 58, 34: 59, 35: 60, 36: 61, 37: 62, 38: 63, 39: 64, 40: 65, 41: 66, 42: 67, 43: 68, 44: 69, 45: 70, 46: 71, 47: 72, 48: 73, 49: 74, 50: 75, 51: 76, 52: 77, 53: 78, 54: 79, 55: 80, 56: 81, 57: 82, 58: 83, 59: 84, 60: 85, 61: 86, 62: 87, 63: 88, 64: 89, 65: 90, 66: 91, 67: 92, 68: 93, 69: 94, 70: 95, 71: 96, 72: 97, 73: 98, 74: 99, 75: 100, 76: 101, 77: 102, 78: 103, 79: 104, 80: 105}

In [ ]:
#sns.distplot(network_output.cpu().numpy())

In [ ]:
#network_input.shape

In [ ]:
# #network_input.cpu()
# network_output.cpu()

In [ ]:
print(network_output.max())
print(network_output.min())

In [ ]:
#torch.save(network_input, 'network_input.pt')
#torch.save(network_output, 'network_output.pt')

In [ ]:
print(network_input.max())
print(network_input.min())
print(max_midi_number)
print(min_midi_number)
print(int_to_note)

In [ ]:
# '''
# data is highly unbalanced
# # '''
# sns.distplot(torch.tensor(network_output).cpu())
# xx = pd.DataFrame(torch.tensor(network_output).cpu())
# xx.groupby(0).size().to_frame(name='values')

In [ ]:
# '''
# to make batch of equal sizes
# Quick Fix
# '''
# network_input = network_input[: -29]
# network_output = network_output[: -29]

# print(network_input.shape)
# print(network_output.shape)

In [ ]:
#network_input[0]

In [ ]:
input_length = 50
n_notes_Y = n_hidden_units = len(sorted_notes)

model = Sequential()
model.add(LSTM(
    256,
    input_shape=((input_length,1)),
    return_sequences=True
))
model.add(LSTM(n_hidden_units, return_sequences=False))
#model.add(Dropout(0.3))
model.add(Dense(n_hidden_units))
model.add(Activation('softmax'))
rmsprop = optimizers.RMSprop(lr=0.002, rho=0.9, epsilon=None, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=['accuracy'])
# Load the weights to each node
#model.load_weights('./hijacked_weights/10LSTM256-LSTM-400-0.2227.hdf5')
print(model.summary())

In [ ]:
#Remember to change the name of the log file
class live_plot(callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.index = 0
        self.epochs = []
        
        self.losses = []
        self.val_losses = []
        
        self.acc = []
        self.val_acc = []
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.epochs.append(self.index)
        
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        
        self.acc.append(logs.get('acc'))
        self.val_acc.append(logs['val_acc'])
        self.index += 1
        
        clear_output(wait=True)
        plt.plot(self.epochs, self.losses, label="loss")
        plt.plot(self.epochs, self.val_losses, label="val_loss")
        plt.legend()
        plt.show();
        
        plt.plot(self.epochs, self.acc, label="acc")
        plt.plot(self.epochs, self.val_acc, label="val_acc")
        plt.legend()
        plt.show();
        
class dump_to_file(callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        with open('logs/forplot_normalized_log.txt','wb') as f:
            np.savetxt(f,np.array([logs.get('loss'),logs.get('val_loss'),logs.get('acc'),logs.get('val_acc')]))

In [ ]:
filepath = "./weights/clementi_normal/forplot{epoch:02d}.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)

plot = live_plot()
dump = dump_to_file()

callbacks_list = [checkpoint, plot, dump]     
fit_data = model.fit(X.reshape(n, input_length, 1), Y, validation_split=0.20,epochs=400, batch_size=64, callbacks=callbacks_list)

In [ ]:
print(fit_data.history.keys())
# summarize history for accuracy
#plt.figure(figsize=(15,10))
plt.plot(fit_data.history['acc'])
plt.plot(fit_data.history['val_acc'])
plt.title('LSTM accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('accuracy_training.pdf')
plt.show()
# summarize history for loss
#plt.figure(figsize=(15,10))
plt.plot(fit_data.history['loss'])
plt.plot(fit_data.history['val_loss'])
plt.title('LSTM loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower left')
plt.savefig('loss_training.pdf')
plt.show()

## MUSIC GENERATION

In [ ]:
#load weights
test_model = Stacked_LSTM(input_size,hidden_size,num_layer,output_size)
test_model.load_state_dict(torch.load('lstm_state_256-38-removed_Ht_Ct_sorted.pt'))
test_model.eval()
test_model.cuda()

In [ ]:
#load population database
testing_data = np.ones(200)*0
testing_data = list(range(24,105))
testing_data.extend(testing_data[::-1])
testing_data_rev = testing_data[::-1]
testing_data_rev.extend(testing_data)
testing_data_rev.extend(testing_data_rev)
testing_data = testing_data_rev


testing_data = np.asarray(testing_data)
testing_data = testing_data.reshape(testing_data.shape[0],1)

initial_seq = [network_input[0][1:].cpu().numpy().tolist()]

testing_data_unnorm = testing_data

In [ ]:
list1=[]
testing_data=testing_data.tolist()
for i in range(len(testing_data)):
    list1.extend(testing_data[i])

#list1

for i in range(len(list1)):
    list1[i]=(list1[i]-24)/(105-24)

list1 = np.asarray(list1)
list1 = list1.reshape(list1.shape[0],1)
testing_data = list1
#list1

In [ ]:
#testing_data

In [ ]:
#Predicting
def prediction_with_influence(influence,int2note,initial_seq, max_note, test_batch_size = 1):

    predicted_notes = []
    initial_seq[0].extend([[0]]*len(testing_data))
    test_seq = torch.Tensor(initial_seq).cuda()
    
    
    for i in range(len(influence)):
        
        test_seq[0][sequence_length - 1 + i][0] = float(influence[i])
        
        test_slice = test_seq[0][i : i + sequence_length]        
        test_slice = test_slice.view(1, test_slice.shape[0], test_slice.shape[1])
                
        
        test_output = test_model.forward(test_slice, test_batch_size)
    
        test_output = F.softmax(test_output, dim = 1)
        top_p, top_class = test_output.topk(1,dim =1)
        test_seq[0][sequence_length - 1 + i][0] = int2note[top_class.item()]/max_note
        
        predicted_notes.append(int2note[top_class.item()])
        
    return predicted_notes

In [ ]:
predicted_notes_lst = prediction_with_influence(testing_data,int_to_note,initial_seq, max_midi_number)

In [ ]:
#predicted_notes_lst

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

#plt.plot(list(np.ones(200)*24))

#plt.plot(list(np.ones(200)*50))
#plt.plot(list(np.ones(20)*50))
plt.plot(testing_data_unnorm)
plt.plot(predicted_notes_lst)

In [ ]:
from collections import Counter
c = Counter(predicted_notes_lst)
c